# 주어진 Crop 영역을 새로운 이미지 파일로 만들기 위한 작업

## data prepration

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [54]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import cv2

from tqdm import tqdm

import os
import warnings
warnings.filterwarnings(action='ignore')

In [6]:
def load_image_path(dir):
    food_middle_list = sorted(os.listdir(dir))

    food_list = [] # 음식이름
    food_path_list = [] # 음식이름 경로

    for food_middle in food_middle_list:
        middle_path = os.path.join(dir,food_middle)
        food_name_list = os.listdir(middle_path)

        for food in food_name_list:
            path = os.path.join(middle_path,food)
            food_path_list.append(path)

            name = path.split('/')[-1]
            food_list.append(name)

    return food_list, food_path_list


In [7]:
data_dir = '/content/drive/MyDrive/project3/data/traindata/kfood'
food_list, food_path_list = load_image_path(data_dir)

print(len(food_list))
print(food_list)
print(food_path_list)

99
['갈비구이', '갈치구이', '고등어구이', '곱창구이', '닭갈비', '장어구이', '조개구이', '떡갈비', '불고기', '삼겹살', '더덕구이', '조기구이', '황태구이', '훈제오리', '콩나물국', '계란국', '떡국_만두국', '시래기국', '무국', '북엇국', '미역국', '육개장', '과메기', '편육', '콩자반', '양념치킨', '피자', '후라이드치킨', '젓갈', '갓김치', '깍두기', '나박김치', '무생채', '배추김치', '부추김치', '열무김치', '백김치', '총각김치', '오이소박이', '파김치', '가지볶음', '고사리나물', '미역줄기볶음', '숙주나물', '시금치나물', '애호박볶음', '송편', '꿀떡', '경단', '만두', '장', '적', '장아찌', '전', '전골', '조림', '라면', '쫄면', '짬뽕', '물냉면', '짜장면', '비빔냉면', '수제비', '막국수', '잔치국수', '열무국수', '칼국수', '콩국수', '도토리묵', '콩나물무침', '꽈리고추무침', '도라지무침', '잡채', '홍어무침', '회무침', '김밥', '김치볶음밥', '누룽지', '알밥', '비빔밥', '자

## Crop images

In [8]:
# 경로 생성

# crop file - crop_area.properties

In [14]:
file_path = crop_files[0]

# crop area -> file_name = area 의 형태
# Img_039_0036=5,333,575,216
# 딕셔너리 형태로 변환 file_name(key) : area(value)
crop_areas = {}

# 파일 읽기
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        # 각 줄에서 = 을 기준으로 키-값 쌍 분리
        key, value = line.strip().split('=')
        # 값을 정수 리스트로 변환
        coordinates = list(map(int, value.split(',')))
        # 딕셔너리에 저장
        crop_areas[key] = coordinates

for key, value in list(crop_areas.items())[:1]:
    print(f"{key}: {value}")

Img_000_0001: [45, 76, 538, 404]


In [37]:


crop_file_name = '/crop_area.properties'
crop_file_path = raw_image_path + crop_file_name

crop_image_path = raw_image_path.replace('kfood', 'crop_image') + '/'

print(raw_image_path)
print(crop_file_path)
print(crop_image_path)

/content/drive/MyDrive/project3/data/traindata/kfood/구이/갈비구이
/content/drive/MyDrive/project3/data/traindata/kfood/구이/갈비구이/crop_area.properties
/content/drive/MyDrive/project3/data/traindata/crop_image/구이/갈비구이/


In [52]:
def make_new_image(raw_image_path):
    crop_file_name = 'crop_area.properties'
    crop_file_path = os.path.join(raw_image_path, crop_file_name)

    boxed_image_path = raw_image_path.replace('kfood', 'boxed_image')
    crop_image_path = raw_image_path.replace('kfood', 'crop_image')

    # 폴더가 존재하지 않으면 생성
    if not os.path.exists(boxed_image_path):
        os.makedirs(boxed_image_path)
    if not os.path.exists(crop_image_path):
        os.makedirs(crop_image_path)

    # 딕셔너리 초기화
    crop_areas = {}

    # 파일 읽기
    with open(crop_file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

        for line in tqdm(lines, desc="Processing images"):
            # 각 줄에서 = 을 기준으로 키-값 쌍 분리
            key, value = line.strip().split('=')
            # 값을 정수 리스트로 변환
            coordinates = list(map(int, value.split(',')))
            # 딕셔너리에 저장
            crop_areas[key] = coordinates

            # 이미지 파일 경로 - jpg, png, bmp 확장자 확인
            img_extensions = ['.jpg', '.png', '.bmp']
            image_path = None

            for ext in img_extensions:
                potential_path = os.path.join(raw_image_path, f"{key}{ext}")
                if os.path.exists(potential_path):
                    image_path = potential_path
                    break

            # 이미지 읽기
            if image_path is not None and os.path.isfile(image_path):
                image = cv2.imread(image_path)

                if image is not None:
                    x, y, w, h = coordinates

                    # Save Cropped image
                    ## Crop image
                    crop_image = image[y:y+h, x:x+w]
                    ## save
                    output_image_path = os.path.join(crop_image_path, f"crop_{key}.jpg")
                    cv2.imwrite(output_image_path, crop_image)

                    # Save Boxed image
                    ## Draw Box
                    boxed_image = cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)
                    ## save
                    output_image_path = os.path.join(boxed_image_path, f"boxed_{key}.jpg")
                    cv2.imwrite(output_image_path, boxed_image)
            #     else :
            #         print(f"Image can't read: {image_path}")
            # else :
            #     print(f"Image not found: {image_path}")



In [55]:

make_crop_image(raw_image_path)

Processing images:   6%|▋         | 14/224 [00:00<00:06, 31.81it/s]

Image not found: None
Image not found: None
Image not found: None


Processing images:  11%|█         | 24/224 [00:00<00:05, 35.43it/s]

Image not found: None
Image not found: None
Image not found: None


Processing images: 100%|█████████▉| 223/224 [00:08<00:00, 24.12it/s]

Image not found: None
Image not found: None
Image not found: None


Processing images: 100%|██████████| 224/224 [00:08<00:00, 26.72it/s]


In [50]:
def drawing_crop_image(raw_image_path):
    crop_file_name = 'crop_area.properties'
    crop_file_path = os.path.join(raw_image_path, crop_file_name)

    crop_image_path = raw_image_path.replace('kfood', 'boxed_image')

    # 폴더가 존재하지 않으면 생성
    if not os.path.exists(crop_image_path):
        os.makedirs(crop_image_path)

    # 딕셔너리 초기화
    crop_areas = {}

    # 파일 읽기
    with open(crop_file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # 각 줄에서 = 을 기준으로 키-값 쌍 분리
            key, value = line.strip().split('=')
            # 값을 정수 리스트로 변환
            coordinates = list(map(int, value.split(',')))
            # 딕셔너리에 저장
            crop_areas[key] = coordinates

            # 이미지 파일 경로 - jpg, png, bmp 확장자 확인
            possible_extensions = ['.jpg', '.png', '.bmp']
            image_path = None

            for ext in possible_extensions:
                potential_path = os.path.join(raw_image_path, f"{key}{ext}")
                if os.path.exists(potential_path):
                    image_path = potential_path
                    break

            # 이미지 읽기
            if image_path is not None and os.path.isfile(image_path):
                image = cv2.imread(image_path)
                if image is not None:
                    # 박스 그리기 (x, y, w, h)
                    x, y, w, h = coordinates
                    boxed_image = cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)

                    # 박스를 그린 이미지 저장
                    output_image_path = os.path.join(crop_image_path, f"crop_{key}.jpg")
                    cv2.imwrite(output_image_path, boxed_image)




In [48]:
with open(crop_file_path, 'r', encoding='utf-8') as file:
    for line in file:
        # 각 줄에서 = 을 기준으로 키-값 쌍 분리
        key, value = line.strip().split('=')
        # 값을 정수 리스트로 변환
        coordinates = list(map(int, value.split(',')))
        # 딕셔너리에 저장
        crop_areas[key] = coordinates